# Installation

Installation performed using Python 3.8.19 + WSL 2 on Windows platform. It is highly possible to do that on Linux. The main issue can be lib version differences.

## Prerequisities

When performed on WLSL:
* Install WSLS Cuda: https://www.youtube.com/watch?v=JaHVsZa2jTc

Install libs (versions may change especially if MMOCR upgrades from 1.0.1 - it was tested on 1.0.1):

In [2]:
%pip install "torch==2.0.0+cu118" "torchvision==0.15.1+cu118" -f https://download.pytorch.org/whl/torch_stable.html
%pip install -U openmim
!mim install "mmengine>=0.7.1,<1.1.0"
!mim install "mmcv>=2.0.0rc4,<2.1.0"
!mim install "mmdet>=3.0.0rc5,<3.2.0"

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 55.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1
    Uninstalling torchvision-0.15.1:
      Successfully uninstalled torchvision-0.15.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html


Install nvidia-cuda-toolkit: `apt install nvidia-cuda-toolkit`

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


## MMOCR

### Clone and install from github

Do not download it as lib, MMOCR states that it should be compiled from source!

In [7]:
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
%pip install -v -e .
# "-v" increases pip's verbosity.
# "-e" means installing the project in editable mode,
# That is, any local modifications on the code will take effect immediately.

Cloning into 'mmocr'...
remote: Enumerating objects: 16384, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 16384 (delta 101), reused 200 (delta 70), pack-reused 16091
Receiving objects: 100% (16384/16384), 16.24 MiB | 21.76 MiB/s, done.
Resolving deltas: 100% (10769/10769), done.
/home/kaybenot/MMOCR/mmocr
Using pip 23.3.1 from /home/kaybenot/MMOCR/.conda/lib/python3.8/site-packages/pip (python 3.8)
Obtaining file:///home/kaybenot/MMOCR/mmocr
  Preparing metadata (setup.py) ...   Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-dynf1grr/mmocr.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-dynf1grr/mmocr.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-dynf1grr/mmocr.egg-info/SOURCES.txt'
done
  Running setup.py develop for mmocr
    Running command python setup.py develop
    running develop
    /home/kaybenot/MMOCR/.conda/lib/python3.8/site-p

### Test if installed correcty

General test:

In [1]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check mmocr installation
import mmocr
print(mmocr.__version__)

!ls ./mmocr/

2.0.0+cu118 True
3.1.0
2.0.1
11.7
GCC 9.3
1.0.1
CITATION.cff	 configs      docker	       projects		 setup.py
LICENSE		 data	      docs	       requirements	 tests
MANIFEST.in	 dataset_zoo  mmocr	       requirements.txt  tools
README.md	 demo	      mmocr.egg-info   resources
README_zh-CN.md  dicts	      model-index.yml  setup.cfg


CUDA test (if any error occures, there is high possibility that CUDA is not installed):

In [2]:
import torch

print(f"{torch.cuda.is_available()=}")

f = torch.nn.Conv2d(3, 8, 3, device="cuda")
X = torch.randn(2, 3, 4, 4, device="cuda")

Y = X @ X
print(f"{Y.shape=}")
print("matrix multiply works")

Y = f(X)
print(f"{Y.shape=}")
print("Conv2d works")

torch.cuda.is_available()=True
Y.shape=torch.Size([2, 3, 4, 4])
matrix multiply works
Y.shape=torch.Size([2, 8, 2, 2])
Conv2d works


MMOCR demo - it should output Json with **predictions**, **recognized texts**, **recognition scores** and **text detection polygons**:

In [3]:
from mmocr.apis import MMOCRInferencer
ocr = MMOCRInferencer(det='DBNet', rec='CRNN')
ocr('demo/demo_text_ocr.jpg', show=True, print_result=True)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textdet/dbnet/dbnet_resnet50-oclip_1200e_icdar2015/dbnet_resnet50-oclip_1200e_icdar2015_20221102_115917-bde8c87a.pth
06/05 22:05:23 - mmengine - WARNING - Failed to search registry with scope "mmocr" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmocr" is a correct scope, or whether the registry is initialized.
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/crnn/crnn_mini-vgg_5e_mj/crnn_mini-vgg_5e_mj_20220826_224120-8afbedbb.pth


/home/kaybenot/MMOCR/.conda/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



Output()

{'predictions': [{'rec_texts': ['cbanks', 'jocece', 'a', 'grourwats', 'pobinsonsg', 'sokyer', 'choh', 'cil', 
'ugranoet', '70', 'round', 'sile', 's', 'lyear', 'ally', 'sue', 'sall'], 'rec_scores': [0.8576307992140452, 
0.9604392548402151, 0.12790250778198242, 0.6551561852296194, 0.7049362927675247, 0.43728799124558765, 
0.37408775091171265, 0.23857185741265616, 0.570589903742075, 0.9830673635005951, 0.9999836444854736, 
0.7943202257156372, 0.2234356701374054, 0.9995116353034973, 0.9990823864936829, 0.6866671641667684, 
0.9515504688024521], 'det_polygons': [[558.7865821116255, 415.0560796260834, 559.2102916827417, 385.82961320877075,
621.0311547550155, 386.7252767086029, 620.6074451838991, 415.9517431259155], [466.11016971289865, 405.838788151741,
470.2080413692773, 373.06595385074615, 559.7558041168338, 384.25603926181793, 555.6579324604553, 
417.0288735628128], [656.6650205761317, 322.275390625, 656.6650205761317, 318.701171875, 663.2197530864197, 
318.701171875, 663.2197530864197, 322.275390625], [591.117695473251, 328.828125, 591.117695473251, 312.1484375, 
678.7127572016461, 312.1484375, 678.7127572016461, 328.828125], [501.5573487332819, 320.20148277282715, 
503.45454925411525, 303.26722890138626, 590.2374706629372, 312.98396348953247, 588.3402701421039, 329.91823554039],
[449.74177798675413, 311.0096424818039, 452.6934152359825, 295.18298774957657, 501.632416248714, 304.3044191598892,
498.6807789994856, 320.1310920715332], [621.2776700745885, 289.07786697149277, 677.3888921039095, 
288.2014009356499, 677.726841403035, 309.82758045196533, 621.615619373714, 310.7040464878082], [581.191653203768, 
307.06714510917664, 582.0860622829861, 287.3954164981842, 623.6578631365741, 289.2844760417938, 622.763454057356, 
308.95620465278625], [518.3352293917181, 298.8758999109268, 520.6605038740998, 281.124826669693, 582.5850212191358,
289.2315012216568, 580.2597467367541, 306.9825744628906], [22.643621399176954, 217.431640625, 44.09547325102881, 
217.431640625, 44.09547325102881, 234.70703125, 22.643621399176954, 234.70703125], [447.50775563432353, 
219.9731957912445, 451.2199606561857, 175.01175343990326, 664.2210897312243, 192.58743166923523, 660.5088847093622,
237.54887402057648], [18.07533664781861, 113.18106681108475, 19.715992843565136, 87.59459227323532, 
84.18860044126157, 91.72619193792343, 82.54793515303498, 117.31266647577286], [589.3300411522633, 94.716796875, 
589.3300411522633, 91.73828125, 592.3094650205761, 91.73828125, 592.3094650205761, 94.716796875], 
[571.3531533323688, 113.65734934806824, 575.8138513133359, 67.87442460656166, 709.5746712641461, 80.89909493923187,
705.113973283179, 126.68201968073845], [454.50100027327676, 105.97232818603516, 457.00514202353395, 
60.077272802591324, 593.2632298096707, 67.50726401805878, 590.7590880594136, 113.40231940150261], 
[31.859713732558514, 56.83516018092632, 32.7709257149402, 39.07187946140766, 70.36558486247749, 40.99922485649586, 
69.4543728800958, 58.76250557601452], [157.76203954877187, 39.99610282480717, 159.33489312065973, 
12.086636293679476, 218.0822797108089, 15.39533443748951, 216.50942613892104, 43.30480210483074]], 'det_scores': 
[0.9016104936599731, 0.8365001082420349, 0.33921384811401367, 0.7745375037193298, 0.816601037979126, 
0.7732779383659363, 0.7893370985984802, 0.8041045069694519, 0.8527858257293701, 0.39253732562065125, 
0.8736308217048645, 0.9159561991691589, 0.33609625697135925, 0.906568706035614, 0.9165750741958618, 
0.8257052302360535, 0.8489266633987427]}]}

{'predictions': [{'rec_texts': ['cbanks',
    'jocece',
    'a',
    'grourwats',
    'pobinsonsg',
    'sokyer',
    'choh',
    'cil',
    'ugranoet',
    '70',
    'round',
    'sile',
    's',
    'lyear',
    'ally',
    'sue',
    'sall'],
   'rec_scores': [0.8576307992140452,
    0.9604392548402151,
    0.12790250778198242,
    0.6551561852296194,
    0.7049362927675247,
    0.43728799124558765,
    0.37408775091171265,
    0.23857185741265616,
    0.570589903742075,
    0.9830673635005951,
    0.9999836444854736,
    0.7943202257156372,
    0.2234356701374054,
    0.9995116353034973,
    0.9990823864936829,
    0.6866671641667684,
    0.9515504688024521],
   'det_polygons': [[558.7865821116255,
     415.0560796260834,
     559.2102916827417,
     385.82961320877075,
     621.0311547550155,
     386.7252767086029,
     620.6074451838991,
     415.9517431259155],
    [466.11016971289865,
     405.838788151741,
     470.2080413692773,
     373.06595385074615,
     559.755804116833